Executar MANUALMENTE os passos abaixo:
1. Executar a transação ZL1PM-COCKPIT
2. Selecionar o Layout OLEO
3. Executar a pesquisa (F8)
4. Exportar o resultado para a pasta C:\Users\Z0038362\OneDrive - ZF Friedrichshafen AG\Documents\Python Scripts
5. Salvar com o nome notasAbertas
6. Clicar em Permitir nas mensagens que forem abrindo
7. Adicionar as colunas "Oleo" e "Qtd" no Excel aberto
8. Salvar e fechar o Excel Aberto

In [6]:
### Importar bibliotecas necessárias ###
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pyautogui as pag

In [7]:
### Carregar a planilha preenchida com as trocas de óleo
planilhaDf_notas = pd.read_excel('notasAbertas.xlsx')

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [8]:
planilhaDf_notas["Data da nota"] = pd.to_datetime(planilhaDf_notas["Data da nota"])

### Avaliar se leu o arquivo corretamente
planilhaDf_notas.head()

,Data da nota,Sala,Ordem,Nota,Equipamento,Descrição,Oleo,Qtd
0,2022-02-28,J020,NaN,2688717,159698,Abastecimento de Oleo GC,GC,40
1,2022-02-28,F030,NaN,2688718,159612,Abastecimento de Oleo GC,GC,50
2,2022-02-28,F040,NaN,2688719,159613,Abastecimento de Oleo GC,GC,45
3,2022-02-28,B010,NaN,2688720,159494,Abastecimento de Oleo GC,GC,60
4,2022-02-28,F010,NaN,2688721,159608,Abastecimento de Oleo GC,GC,40


In [9]:
### Processo de Login com SAP WebGUI ###

## Abre janela para digitar usuário e senha
usuario = pag.prompt('Digita seu usuário')
senha = pag.prompt('Digita sua senha')

## criar o navegador com o webdriver do Chrome
navegador = webdriver.Chrome()

## Link de acesso do WebGUI do SAP da ZF
navegador.get("https://swdlp1.zf-world.com/sap/bc/gui/sap/its/webgui")

## Preencher as informações de Login e senha
navegador.find_element(By.XPATH, '//*[@id="sap-user"]').send_keys(usuario)
navegador.find_element(By.XPATH, '//*[@id="sap-password"]').send_keys(usuario)

## Clicar no botão para realizar o LOGIN
navegador.find_element(By.XPATH, '//*[@id="LOGON_BUTTON"]').click()

In [10]:
time.sleep(10)
navegador.switch_to.frame("ITSFRAME1");
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys("IW34")
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys(Keys.ENTER)

In [11]:
pos = 0
tamanhoPlaniha = len(planilhaDf_notas)
wait = WebDriverWait(navegador, 10)

for i in planilhaDf_notas.Nota:
    time.sleep(2)
    ## Aguardar a tela carregar e digitar o tipo de manutenção PM03, colocar o código da nota e pressionar ENTER
    fastrack = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="M0:46:::2:17"]')))
    fastrack.send_keys("PM03")
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::4:17"]').clear()
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::4:17"]').send_keys(str(planilhaDf_notas.Nota[pos]))
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::4:17"]').send_keys(Keys.ENTER)
    time.sleep(1)
    
    ## Preencher a data de término planejada
    fastrack2 = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@title="Data-base do fim"]')))
    fastrack2.send_keys(str(planilhaDf_notas['Data da nota'][pos].strftime("%d.%m.20%y")))
    navegador.find_element(By.XPATH, '//*[@title="Data-base do fim"]').send_keys(Keys.ENTER)
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="M0:46:1:1:2:3B256:5::1:12"]').send_keys(Keys.ENTER)
    time.sleep(2)
    navegador.find_element(By.XPATH, '//*[@id="M0:46:1:1:2:3B256:5::1:12"]').send_keys(Keys.ENTER)
    time.sleep(2)
    
    ## Clicar na aba Componentes
    navegador.find_element(By.XPATH, '//*[@id="M0:46:1:1:2::0:3-title"]').click()
    time.sleep(3)
    
    ## Aguarda abrir a janela componentes e digitar os campos
    if planilhaDf_notas['Oleo'][pos] == 'GC':
        pag.write('AA02.421.024')
    elif planilhaDf_notas['Oleo'][pos] == 'SSM20':
        pag.write('AA02.421.150')
    pag.press('tab', presses=2)
    pag.write(str(planilhaDf_notas['Qtd'][pos]))
    pag.press('tab', presses=2)
    pag.write('L')
    pag.press('enter')
    time.sleep(3)
    pag.press('enter')
    time.sleep(3)
    
    ## Clicar no botão "Por em processo"
    navegador.find_element(By.XPATH, '//*[@id="M0:37::btn[26]"]').click()
    time.sleep(3)
    
    ## Selecionar para não imprimir a nota
    pag.press('tab')
    time.sleep(1)
    pag.press('tab')
    time.sleep(2)
    pag.press('right')
    time.sleep(1)
    pag.press('right')
    time.sleep(2)
    pag.press('enter')

    pos = pos + 1
    if pos > tamanhoPlaniha:
        pag.alert("Finalizado abertura das notas!")
        break